In [ ]:
!pip install ultralytics -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.2/882.2 kB 48.6 MB/s eta 0:00:00


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from ultralytics import YOLO

class AutonomousVehicleAI:
    def __init__(self, yolo_model_path):
        # Load the fine-tuned YOLOv8n model for known object detection
        self.known_object_model = YOLO(yolo_model_path)

        # Load or define an autoencoder for novel object detection
        self.novel_object_model = self.build_autoencoder()

        # Placeholder for collision avoidance model (or logic)
        self.collision_avoidance_model = None

    def build_autoencoder(self):
        # Build a simple autoencoder for anomaly detection
        input_img = tf.keras.layers.Input(shape=(224, 224, 3))

        # Encoder
        x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
        x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
        x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
        encoded = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)

        # Decoder
        x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
        x = tf.keras.layers.UpSampling2D((2, 2))(x)
        x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        x = tf.keras.layers.UpSampling2D((2, 2))(x)
        decoded = tf.keras.layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

        # Compile the model
        autoencoder = tf.keras.Model(input_img, decoded)
        autoencoder.compile(optimizer='adam', loss='mean_squared_error')

        return autoencoder

    def detect_known_objects(self, frame):
        # Perform detection using YOLO
        results = self.known_object_model(frame)

        # Extract the results
        detected_objects = []
        for result in results:
            for box in result.boxes:  # Each result contains boxes
                detected_objects.append({
                    'class_id': box.cls.item(),  # Class ID
                    'confidence': box.conf.item(),  # Confidence score
                    'bbox': box.xyxy[0].cpu().numpy()  # Bounding box coordinates
                })

        return detected_objects

    def detect_novel_objects(self, frame):
        # Preprocess the frame
        img = cv2.resize(frame, (224, 224))
        img = np.expand_dims(img, axis=0) / 255.0  # Normalize the image for autoencoder

        # Predict using the autoencoder (reconstruct the image)
        reconstructed = self.novel_object_model.predict(img)

        # Calculate reconstruction error (Mean Squared Error)
        reconstruction_error = np.mean(np.square(img - reconstructed))

        # Set a threshold for novelty detection (tune this based on validation set)
        novelty_threshold = 0.02

        # Detect if the object is novel based on the reconstruction error
        if reconstruction_error > novelty_threshold:
            return True
        else:
            return False

    def make_collision_avoidance_decision(self, detected_objects, sensor_data):
        # collision avoidance logic
        for obj in detected_objects:
            if isinstance(obj, bool) and obj:  # If novel object detected
                print("Novel object detected! Taking evasive action.")
                return "Evasive action taken"
            elif obj['confidence'] > 0.5:  # If high confidence in known object detection
                print(f"Known object detected: {obj['class_id']} with confidence {obj['confidence']}")

        # Use sensor data for additional decisions (e.g., slowing down, stopping)
        if sensor_data.get("distance_to_object", 100) < 10:
            return "Slowing down or stopping"

        return "Proceed with caution"

    def process_frame(self, frame, sensor_data):
        # Detect known objects
        known_objects = self.detect_known_objects(frame)

        # Detect novel objects
        novel_object_detected = self.detect_novel_objects(frame)

        # Combine the known and novel objects
        decision = self.make_collision_avoidance_decision(known_objects + [novel_object_detected], sensor_data)

        return decision

### Example code usage

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Initialize your AI system with the path to the fine-tuned YOLOv8n model
vehicle_ai = AutonomousVehicleAI('/content/drive/MyDrive/Colab Notebooks/Interview Test/yolov8n-kitti/train/weights/best.pt')

# Example frame and sensor data
frame = np.random.rand(224, 224, 3) * 255
sensor_data = {"distance_to_object": 8}

# Process the frame and make a decision
decision = vehicle_ai.process_frame(frame, sensor_data)
print("Decision:", decision)


0: 640x640 (no detections), 7.3ms
Speed: 81.7ms preprocess, 7.3ms inference, 139.7ms postprocess per image at shape (1, 3, 640, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Novel object detected! Taking evasive action.
Decision: Evasive action taken


## Example video input

In [ ]:
model_path = '/content/drive/MyDrive/Colab Notebooks/Interview Test/yolov8n-kitti/train/weights/best.pt'
ai_system = AutonomousVehicleAI(model_path)

# Simulating video input (replace with actual video capture)
video = cv2.VideoCapture("/content/drive/MyDrive/Colab Notebooks/Interview Test/input.mp4")

while True:
    ret, frame = video.read()
    if not ret:
        break

    # Simulating sensor data (replace with actual sensor inputs)
    sensor_data = {"distance_to_object": 8}

    decision = ai_system.process_frame(frame, sensor_data)
    print("Decision:", decision)


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

Streaming output truncated to the last 5000 lines.

0: 384x640 (no detections), 18.0ms
Speed: 4.9ms preprocess, 18.0ms inference, 5.3ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Novel object detected! Taking evasive action.
Decision: Evasive action taken

0: 384x640 (no detections), 18.0ms
Speed: 3.9ms preprocess, 18.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Novel object detected! Taking evasive action.
Decision: Evasive action taken

0: 384x640 (no detections), 22.2ms
Speed: 9.2ms preprocess, 22.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Novel object detected! Taking evasive action.
Decision: Evasive action taken

0: 384x640 (no detections), 9.2ms
Speed: 3.4ms preprocess, 9.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Novel object detected! Taking evasiv